In [1]:
import pandas as pd
import numpy as np
from itertools import product

In [2]:
sales = pd.read_csv('/Users/sinsakuokazaki/Project/pred-future-sales/\
input/competitive-data-science-predict-future-sales/sales_train.csv')


In [3]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,dayofweek,dayofmonth,month,quarter,year,minute,hour,total_sales,shop_name,item_name,item_category_id,item_category_name
0,2013-01-02,0,59,22154,999.00,1.0,2,2,1,1,2013,0,0,999.00,"Ярославль ТЦ ""Альтаир""",ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
1,2013-01-03,0,25,2552,899.00,1.0,3,3,1,1,2013,0,0,899.00,"Москва ТРК ""Атриум""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
2,2013-01-05,0,25,2552,899.00,-1.0,5,5,1,1,2013,0,0,-899.00,"Москва ТРК ""Атриум""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
3,2013-01-06,0,25,2554,1709.05,1.0,6,6,1,1,2013,0,0,1709.05,"Москва ТРК ""Атриум""",DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил
4,2013-01-15,0,25,2555,1099.00,1.0,1,15,1,1,2013,0,0,1099.00,"Москва ТРК ""Атриум""",DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства


In [4]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})

#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

/anaconda3/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4658: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [28]:
from sklearn.model_selection import KFold
y = all_data["target"].values
X = all_data.drop(["target"], axis = 1).values
kfl = KFold(5, shuffle=False, random_state=123)

In [29]:
all_data_new = all_data.copy()
all_data_new['item_target_emc'] = np.nan

In [30]:
fold = 0
for tr_ind, val_ind in kfl.split(X, y):
    print("{} fold".format(fold))
    X_tr, X_val = all_data.iloc[tr_ind], all_data.iloc[val_ind]
    means = X_val["item_id"].map(X_tr.groupby("item_id")["target"].transform("mean"))
    all_data_new.loc[val_ind, "item_target_enc"] = means
    fold += 1
all_data_new['item_target_enc'].fillna(0.3343, inplace=True) 
encoded_feature = all_data_new['item_target_enc'].values
# You will need to compute correlation like that
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0 fold
1 fold
2 fold
3 fold
4 fold
-0.006492061332241811


In [40]:
all_data_new2 = all_data.copy()
target_sum = all_data_new2.groupby("item_id")["target"].sum()
n_objects = all_data_new2.groupby("item_id")["target"].count()

In [43]:
n_objects[0]

50

In [42]:
target_sum[0]

1.0

In [39]:
all_data_new2.groupby("item_id")["target"].app

In [49]:
all_data_new2.head()

,shop_id,item_id,date_block_num,target
139255,0,19,0,0.0
141495,0,27,0,0.0
144968,0,28,0,0.0
142661,0,29,0,0.0
138947,0,32,0,6.0


In [55]:
all_data_new2 = all_data.copy()
target_sum = all_data_new2.groupby("item_id")["target"].sum()
n_objects = all_data_new2.groupby("item_id")["target"].count()
all_data_new2["item_target_enc2"] = all_data_new2.apply(lambda x: (target_sum[x['item_id']] - x['target']) \
                                                        / n_objects[x['item_id']], axis = 1) 

In [56]:
encoded_feature =  all_data_new2['item_target_enc2'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4803928066198171


In [59]:
all_data_new4 = all_data.copy()


In [69]:
cumsum = all_data_new4.groupby("item_id")["target"].cumsum() - all_data_new4["target"]

In [70]:
cumsum

139255         0.0
141495         0.0
144968         0.0
142661         0.0
138947         0.0
138948         0.0
138949         0.0
139247         0.0
142672         0.0
142065         0.0
139208         0.0
142670         0.0
139207         0.0
138950         0.0
143764         0.0
141505         0.0
139199         0.0
138952         0.0
139176         0.0
138951         0.0
139177         0.0
139178         0.0
139179         0.0
143769         0.0
142671         0.0
144539         0.0
139180         0.0
138953         0.0
144265         0.0
141744         0.0
             ...  
10772600    1209.0
10770510     126.0
10769953     108.0
10769955    1664.0
10768833     184.0
10769961     420.0
10770625     143.0
10769956     695.0
10771598    2807.0
10767854    3441.0
10768086    5271.0
10768087    1191.0
10768088    2089.0
10767847     389.0
10769954     260.0
10767848    1707.0
10767849     310.0
10768089     104.0
10770726     131.0
10772844     187.0
10771530     121.0
10768090    

In [62]:
cumcnt = all_data_new4.groupby("item_id").cumcount()

In [64]:
cumcnt

139255         0
141495         0
144968         0
142661         0
138947         0
138948         0
138949         0
139247         0
142672         0
142065         0
139208         0
142670         0
139207         0
138950         0
143764         0
141505         0
139199         0
138952         0
139176         0
138951         0
139177         0
139178         0
139179         0
143769         0
142671         0
144539         0
139180         0
138953         0
144265         0
141744         0
            ... 
10772600    1455
10770510     899
10769953     214
10769955    1220
10768833    1124
10769961    1134
10770625     898
10769956     993
10771598    1448
10767854    1585
10768086    1585
10768087    1585
10768088    1585
10767847     171
10769954     258
10767848     258
10767849     351
10768089    1075
10770726    1034
10772844    1585
10771530     498
10768090    1585
10769958     491
10769959     536
10769704      43
10768834     448
10769024     128
10769690     3

In [71]:
from sklearn.model_selection import KFold
y = all_data["target"].values
X = all_data.drop(["target"], axis = 1).values
kfl = KFold(5, shuffle=False, random_state=123)

all_data_new = all_data.copy()
all_data_new['item_target_emc'] = np.nan

fold = 0
for tr_ind, val_ind in kfl.split(X, y):
    print("{} fold".format(fold))
    X_tr, X_val = all_data.iloc[tr_ind], all_data.iloc[val_ind]
    means = X_tr.groupby("item_id")["target"].mean()
    all_data_new.loc[val_ind, "item_target_enc"] = X_val["item_id"].map(means)
    fold += 1
all_data_new['item_target_enc'].fillna(0.3343, inplace=True) 
encoded_feature = all_data_new['item_target_enc'].values

0 fold
1 fold
2 fold
3 fold
4 fold


In [72]:
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4092355966844078
